In [77]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [78]:
theta,beta,M1,gamma = symbols('\\theta \\beta  M_1 \\gamma')
eqn = 2*sp.cot(beta)*((M1**2*sin(beta)-1)/(M1**2*(gamma+cos(2*beta))+2)) - sp.tan(theta)
partial_theta = eqn.diff(theta)
partial_beta = eqn.diff(beta)
partial_mach = eqn.diff(M1)
displayEquations(symbols('\\theta,\\beta,M_1'),eqn)
displayEquations(symbols('\\partial_\\theta'),partial_theta)
displayEquations(symbols('\\partial_\\beta'),partial_beta)
displayEquations(symbols('\\partial_M'),partial_mach)
f_prime = lambdify(args=[theta,beta,M1,gamma],expr=(partial_theta+partial_beta+partial_mach))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [88]:
def ThetaBetaMach(intial_theta_guess_rad,intial_beta_guess_rad,intial_mach_guess,theta_known:bool,beta_known:bool,mach_known:bool):
  theta,beta,M1,gamma = symbols('\\theta \\beta  M_1 \\gamma')
  eqn = 2*sp.cot(beta)*((M1**2*sin(beta)-1)/(M1**2*(gamma+cos(2*beta))+2)) - sp.tan(theta)
  partial_theta = eqn.diff(theta)
  partial_beta = eqn.diff(beta)
  partial_mach = eqn.diff(M1)
  f_prime = lambdify(args=[theta,beta,M1],expr=((partial_theta+partial_beta+partial_mach).subs(gamma,1.4)))
  f = lambdify(args=[theta,beta,M1],expr=(eqn.subs(gamma,1.4)))
  tolerance = 1e-6
  iteration_counter = 0
  iteration_table = {'theta_i+1':[],'beta_i+1':[],'M_i+1':[],'f_i+1':[]}
  if theta_known == False:
    theta_i = intial_theta_guess_rad
    while f(theta_i,intial_beta_guess_rad,intial_mach_guess) > tolerance or  f(theta_i,intial_beta_guess_rad,intial_mach_guess) < -tolerance:
      theta_i = theta_i - f(theta_i,intial_beta_guess_rad,intial_mach_guess)/f_prime(theta_i,intial_beta_guess_rad,intial_mach_guess)
      iteration_table['theta_i+1'].append(theta_i),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(theta_i,intial_beta_guess_rad,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\theta'),theta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return theta_i
  if beta_known == False:
    beta_i = intial_beta_guess_rad
    while f(intial_theta_guess_rad,beta_i,intial_mach_guess) > tolerance or  f(intial_theta_guess_rad,beta_i,intial_mach_guess) < -tolerance:
      beta_i = beta_i - f(intial_theta_guess_rad,beta_i,intial_mach_guess)/f_prime(intial_beta_guess_rad,beta_i,intial_mach_guess)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(beta_i),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,beta_i,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\beta'),beta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return beta_i
  if mach_known == False:
    mach_i = intial_mach_guess
    while f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) > tolerance or  f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) < -tolerance:
      mach_i = mach_i - f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)/f_prime(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(mach_i),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('M_1'),mach_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return mach_i

In [90]:
beta_rad = ThetaBetaMach(16*np.pi/180,10*np.pi/180,1.7,True,False,True)
beta_degrees = beta_rad*180/np.pi
displayEquations(symbols('\\beta'),beta_degrees)

<IPython.core.display.Math object>

,theta_i+1,beta_i+1,M_i+1,f_i+1
0,0.279253,0.407943,1.7,-2.022650e-01
1,0.279253,0.577413,1.7,-4.096791e-02
2,0.279253,0.660020,1.7,7.382058e-03
3,0.279253,0.627560,1.7,-9.996047e-03
4,0.279253,0.657718,1.7,6.212348e-03
...,...,...,...,...
159,0.279253,0.645786,1.7,-1.214148e-06
160,0.279253,0.645790,1.7,1.149746e-06
161,0.279253,0.645786,1.7,-1.088875e-06
162,0.279253,0.645790,1.7,1.031122e-06


<IPython.core.display.Math object>